<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=6f756c9a918c931e23432feaff5b81658a3aff3dc60d895db368bf14ad3aa5bf
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-06 14:23:14
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.24 C
CY Investment:  1.51 C
Reserve:  57.44 K
Current:  1.41 C
-------------------
Today PnL: -18.23 K (-0.13%)
Current PnL: -20.58 L (-13.66%)
CY Booked + Current PnL: -9.14 L (-6.06%)
-------------------
Total profit:  1.42 L
Total loss:  -22.00 L
-------------------
Total Booked + Current PnL: 17.84 L (14.42%)
Total Booked PnL: 38.42 L (31.06%)
Curr Year Booked PnL: 11.44 L (8.11%)
Prev Year Booked PnL: 26.98 L (21.81%)
Est FTT:  2.29 C
Est FTT PnL: 87.78 L (62.25%)
Deployed:  1.24 C
Current:  1.41 C
CAGR/XIRR %: 7.96%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-42.00,63.0,H-MC,13.37,115079.0,14575.0,14534.0,-1.82,14.50,12.63,28.96,79.0,1.00,0.82,53.39,MH,ATH,METALS
77,TTKPRESTIG,770.00,93.17,38.0,M-SC,1.53,83250.0,-17527.0,17616.0,-1.31,-17.39,21.16,0.09,245.0,-0.99,0.59,9.37,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,9.78,48.0,H-MC,2.85,189500.0,24830.0,18495.0,-0.22,15.08,9.76,26.31,89.0,1.34,1.35,24.26,X40N,NTT,AC
42,ITC,452.00,-38.77,46.0,H-LC,2.19,197703.0,-2435.0,23329.0,-0.32,-1.22,11.80,10.44,4.0,-0.10,1.41,4.28,X40,NTT,FMCG
34,ICICIGI,2260.25,-17.09,59.0,H-MC,1.78,142524.0,6391.0,24728.0,0.75,4.69,17.35,22.86,91.0,0.26,1.01,17.90,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,RAJOOENG,143.10,-40.28,44.0,H-SC,16.07,90540.0,-11960.0,52558.0,5.00,-11.67,58.05,39.61,114.0,-0.23,0.64,10.58,AR,ATH,MISC
65,SIEMENS,4671.50,1.88,59.0,H-LC,1.96,163770.0,-22325.0,69799.0,3.52,-12.00,42.62,25.51,15.0,-0.32,1.17,20.69,AR,ATH,ELECTRICAL
48,LTIM,7230.20,-4.34,53.0,H-LC,2.00,237195.0,-8956.0,88165.0,2.95,-3.64,37.17,32.18,16.0,-0.10,1.69,31.09,X200,ATH,IT
4,ANGELONE,3033.00,5.10,50.0,H-SC,11.27,185648.0,-5358.0,63065.0,2.85,-2.81,33.97,30.21,157.0,-0.08,1.32,17.49,X40N,NTT,FINANCE
74,TCS,4406.34,-29.65,44.0,H-LC,12.24,279894.0,-66062.0,134293.0,2.61,-19.10,47.98,19.73,1.0,-0.49,1.99,3.09,X40,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-181.82,62.0,M-SC,11.42,140697.0,-8523.0,152698.0,-5.27,-5.71,108.53,96.62,208.0,-0.06,1.00,52.26,XY25,NTT,FINANCE
66,SIS,528.00,1998.96,31.0,H-SC,3.63,84711.0,-26321.0,49929.0,-3.23,-23.71,58.94,21.26,156.0,-0.53,0.60,14.45,OX40N,NTT,MISC
53,QUESS,986.00,-44.90,35.0,M-SC,38.27,54838.0,-10168.0,161098.0,-2.62,-15.64,293.77,232.18,198.0,-0.06,0.39,0.00,XY24,NTT,MISC
69,SURYODAY,240.00,64.47,66.0,H-SC,5.93,151246.0,-27825.0,93561.0,-2.45,-15.54,61.86,36.71,135.0,-0.30,1.08,50.07,XR,NTT,BANKS
46,KPIGREEN,731.05,-6.33,33.0,H-SC,7.36,111586.0,-13711.0,72642.0,-2.16,-10.94,65.10,47.03,141.0,-0.19,0.79,39.72,MH,ATH,POWER


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,ICICIPRULI,790.0,-21.11,48.0,H-MC,2.13,135781.0,-7.0,42757.0,-0.05,-0.00,31.49,31.48,107.0,-0.00,0.97,12.11,X40,ATH,INSURANCE
86,WIPRO,420.0,-14.70,42.0,M-LC,5.66,149724.0,-1221.0,110676.0,0.21,-0.81,73.92,72.51,53.0,-0.01,1.07,5.16,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,107.70,66.0,H-SC,12.83,130977.0,-11502.0,31028.0,0.35,-8.07,23.69,13.70,163.0,-0.37,0.93,54.59,OX40N,NTT,BANKS
45,KANSAINER,340.0,-67.24,58.0,H-SC,2.01,225810.0,-43857.0,80185.0,-0.58,-16.26,35.51,13.47,138.0,-0.55,1.61,14.82,XY24,NTT,PAINTS
77,TTKPRESTIG,770.0,93.17,38.0,M-SC,1.53,83250.0,-17527.0,17616.0,-1.31,-17.39,21.16,0.09,245.0,-0.99,0.59,9.37,OX40N,NTT,DURABLES
18,CERA,9475.0,-22.67,38.0,H-SC,1.47,140921.0,-34982.0,76999.0,-0.35,-19.89,54.64,23.89,149.0,-0.45,1.00,21.70,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-24.38,42.0,H-MC,6.23,102239.0,-29596.0,71823.0,0.48,-22.45,70.25,32.03,98.0,-0.41,0.73,14.52,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1247.69,69.0,M-SC,20.14,183298.0,2155.0,43313.0,-0.66,1.19,23.63,25.10,235.0,0.05,1.31,46.90,XY24,NTT,HEALTHCARE
60,SAIL,228.00,47.55,60.0,M-MC,12.19,232401.0,7439.0,160217.0,0.03,3.31,68.94,74.53,192.0,0.05,1.65,36.63,XY24,BTT,STEEL
32,HINDZINC,730.22,31.07,67.0,M-LC,5.57,213411.0,8335.0,104230.0,-0.18,4.06,48.84,54.89,52.0,0.08,1.52,29.14,X5K,ATH,METALS
14,BLUESTARCO,2080.00,9.78,48.0,H-MC,2.85,189500.0,24830.0,18495.0,-0.22,15.08,9.76,26.31,89.0,1.34,1.35,24.26,X40N,NTT,AC
84,VOLTAS,1530.00,-2.76,46.0,H-MC,1.64,204075.0,12333.0,25428.0,0.01,6.43,12.46,19.69,99.0,0.49,1.45,13.67,XY25,NTT,AC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-42.00,63.0,H-MC,13.37,115079.0,14575.0,14534.0,-1.82,14.50,12.63,28.96,79.0,1.00,0.82,53.39,MH,ATH,METALS
11,BANKINDIA,190.00,-25.47,67.0,H-MC,10.16,192542.0,12734.0,99679.0,-0.27,7.08,51.77,62.52,88.0,0.13,1.37,43.58,XR,NTT,BANKS
32,HINDZINC,730.22,31.07,67.0,M-LC,5.57,213411.0,8335.0,104230.0,-0.18,4.06,48.84,54.89,52.0,0.08,1.52,29.14,X5K,ATH,METALS
37,INDIAMART,4810.62,-54.64,37.0,H-SC,9.51,126458.0,3122.0,128507.0,0.62,2.53,101.62,106.72,119.0,0.02,0.90,26.50,AR,ATH,MISC
86,WIPRO,420.00,-14.70,42.0,M-LC,5.66,149724.0,-1221.0,110676.0,0.21,-0.81,73.92,72.51,53.0,-0.01,1.07,5.16,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-22.15,30.0,H-LC,9.78,280624.0,-35218.0,147889.0,-0.81,-11.15,52.70,35.67,5.0,-0.24,2.00,1.16,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-18.03,33.0,H-LC,5.81,207002.0,-44766.0,97974.0,-0.23,-17.78,47.33,21.13,27.0,-0.46,1.47,11.35,X40,ATH,PAINTS
19,CIPLA,1795.00,-22.18,43.0,H-LC,4.01,207350.0,2850.0,38567.0,-0.28,1.39,18.60,20.25,10.0,0.07,1.48,10.47,X40N,BTT,PHARMA
30,HCLTECH,1937.36,-2.37,44.0,H-LC,10.00,222500.0,-19424.0,81658.0,1.70,-8.03,36.70,25.73,8.0,-0.24,1.58,5.14,X40,ATH,IT
74,TCS,4406.34,-29.65,44.0,H-LC,12.24,279894.0,-66062.0,134293.0,2.61,-19.10,47.98,19.73,1.0,-0.49,1.99,3.09,X40,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.5,1.88,59.0,H-LC,1.96,163770.0,-22325.0,69799.0,3.52,-12.00,42.62,25.51,15.0,-0.32,1.17,20.69,AR,ATH,ELECTRICAL
48,LTIM,7230.2,-4.34,53.0,H-LC,2.00,237195.0,-8956.0,88165.0,2.95,-3.64,37.17,32.18,16.0,-0.10,1.69,31.09,X200,ATH,IT
42,ITC,452.0,-38.77,46.0,H-LC,2.19,197703.0,-2435.0,23329.0,-0.32,-1.22,11.80,10.44,4.0,-0.10,1.41,4.28,X40,NTT,FMCG
51,NESTLEIND,1377.0,-10.39,51.0,H-LC,3.22,275933.0,10507.0,46274.0,0.80,3.96,16.77,21.40,11.0,0.23,1.96,10.86,XY25,NTT,FMCG
57,RELIANCE,1533.0,-14.41,45.0,H-LC,3.40,214266.0,3900.0,24876.0,0.74,1.85,11.61,13.68,37.0,0.16,1.53,18.68,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-22.15,30.0,H-LC,9.78,280624.0,-35218.0,147889.0,-0.81,-11.15,52.70,35.67,5.0,-0.24,2.00,1.16,X40N,ATH,FMCG
74,TCS,4406.34,-29.65,44.0,H-LC,12.24,279894.0,-66062.0,134293.0,2.61,-19.10,47.98,19.73,1.0,-0.49,1.99,3.09,X40,ATH,IT
42,ITC,452.00,-38.77,46.0,H-LC,2.19,197703.0,-2435.0,23329.0,-0.32,-1.22,11.80,10.44,4.0,-0.10,1.41,4.28,X40,NTT,FMCG
30,HCLTECH,1937.36,-2.37,44.0,H-LC,10.00,222500.0,-19424.0,81658.0,1.70,-8.03,36.70,25.73,8.0,-0.24,1.58,5.14,X40,ATH,IT
80,UNITDSPR,1644.00,-11.23,57.0,H-LC,6.99,232716.0,-1428.0,50057.0,-0.70,-0.61,21.51,20.77,86.0,-0.03,1.66,6.27,X40N,NTT,BREWERIES


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,1.88,59.0,H-LC,1.96,163770.0,-22325.0,69799.0,3.52,-12.00,42.62,25.51,15.0,-0.32,1.17,20.69,AR,ATH,ELECTRICAL
42,ITC,452.00,-38.77,46.0,H-LC,2.19,197703.0,-2435.0,23329.0,-0.32,-1.22,11.80,10.44,4.0,-0.10,1.41,4.28,X40,NTT,FMCG
5,ASIANPAINT,3465.66,-18.03,33.0,H-LC,5.81,207002.0,-44766.0,97974.0,-0.23,-17.78,47.33,21.13,27.0,-0.46,1.47,11.35,X40,ATH,PAINTS
19,CIPLA,1795.00,-22.18,43.0,H-LC,4.01,207350.0,2850.0,38567.0,-0.28,1.39,18.60,20.25,10.0,0.07,1.48,10.47,X40N,BTT,PHARMA
57,RELIANCE,1533.00,-14.41,45.0,H-LC,3.40,214266.0,3900.0,24876.0,0.74,1.85,11.61,13.68,37.0,0.16,1.53,18.68,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4406.34,-29.65,44.0,H-LC,12.24,279894.0,-66062.0,134293.0,2.61,-19.10,47.98,19.73,1.0,-0.49,1.99,3.09,X40,ATH,IT
40,INFY,2275.00,-19.60,46.0,H-LC,8.14,312727.0,-161.0,171843.0,1.49,-0.05,54.95,54.87,3.0,-0.00,2.23,6.59,X40,BTT,IT
42,ITC,452.00,-38.77,46.0,H-LC,2.19,197703.0,-2435.0,23329.0,-0.32,-1.22,11.80,10.44,4.0,-0.10,1.41,4.28,X40,NTT,FMCG
83,VBL,671.64,-22.15,30.0,H-LC,9.78,280624.0,-35218.0,147889.0,-0.81,-11.15,52.70,35.67,5.0,-0.24,2.00,1.16,X40N,ATH,FMCG
1,ABB,7934.00,-40.06,49.0,H-LC,7.57,250512.0,-11107.0,130316.0,0.68,-4.25,52.02,45.57,7.0,-0.09,1.78,6.60,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,46.0,L-SC,32.85,80218.0,-33331.0,73383.0,1.33,-29.35,91.48,35.27,268.0,-0.45,0.57,96.33,XR,NTT,HOTELS
49,MASFIN,398.61,-18.70,47.0,H-SC,11.52,92295.0,-5685.0,27292.0,1.27,-5.80,29.57,22.05,152.0,-0.21,0.66,34.16,XR,ATH,FINANCE
50,NATIONALUM,244.55,-42.00,63.0,H-MC,13.37,115079.0,14575.0,14534.0,-1.82,14.50,12.63,28.96,79.0,1.00,0.82,53.39,MH,ATH,METALS
78,UJJIVANSFB,60.00,107.70,66.0,H-SC,12.83,130977.0,-11502.0,31028.0,0.35,-8.07,23.69,13.70,163.0,-0.37,0.93,54.59,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-181.82,62.0,M-SC,11.42,140697.0,-8523.0,152698.0,-5.27,-5.71,108.53,96.62,208.0,-0.06,1.00,52.26,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SURYODAY,240.00,64.47,66.0,H-SC,5.93,151246.0,-27825.0,93561.0,-2.45,-15.54,61.86,36.71,135.0,-0.30,1.08,50.07,XR,NTT,BANKS
61,SAMMAANCAP,326.00,-181.82,62.0,M-SC,11.42,140697.0,-8523.0,152698.0,-5.27,-5.71,108.53,96.62,208.0,-0.06,1.00,52.26,XY25,NTT,FINANCE
64,SHALBY,327.00,1247.69,69.0,M-SC,20.14,183298.0,2155.0,43313.0,-0.66,1.19,23.63,25.10,235.0,0.05,1.31,46.90,XY24,NTT,HEALTHCARE
32,HINDZINC,730.22,31.07,67.0,M-LC,5.57,213411.0,8335.0,104230.0,-0.18,4.06,48.84,54.89,52.0,0.08,1.52,29.14,X5K,ATH,METALS
11,BANKINDIA,190.00,-25.47,67.0,H-MC,10.16,192542.0,12734.0,99679.0,-0.27,7.08,51.77,62.52,88.0,0.13,1.37,43.58,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.07
1,25,43.27
2,50,74.94


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.10
LC    32.16
MC    23.73
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.92
X40      14.57
X40N     11.22
XR       11.03
XY25     10.33
AR        8.23
OX40N     7.72
X200      1.69
MH        1.61
X5K       1.52
SR        1.15
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.02
H-LC    25.63
H-MC    20.47
M-SC    13.66
M-LC     5.53
M-MC     2.96
L-SC     1.42
L-LC     1.00
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.80,-5.86,40.74
IT,12.65,-18.33,82.43
FINANCE,10.42,-15.31,67.69
MISC,6.94,-17.84,83.13
BANKS,6.35,-11.80,70.62
PAINTS,5.56,-18.28,35.91
ELECTRICAL,5.37,-9.12,47.40
INSURANCE,3.76,-4.76,43.92
AUTO,3.54,-15.80,71.16


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3233267.0,22
XR,1362904.0,14
AR,1172692.0,9
X40,792556.0,10
X40N,630774.0,9
OX40N,560808.0,10
XY25,484806.0,7
SR,260818.0,2
X5K,104230.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3715594.0,29
M-SC,1514157.0,17
H-MC,1229742.0,15
H-LC,1213627.0,15
M-LC,373508.0,4
M-MC,366819.0,2
L-SC,260928.0,3
L-MC,60490.0,1
L-LC,43331.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1219833.0      6
M-SC       XY24       848931.0      7
H-SC       AR         804503.0      5
           XR         789967.0      7
H-MC       XY24       568386.0      4
H-LC       X40        509097.0      5
M-MC       XY24       366819.0      2
H-SC       X40N       309094.0      4
           SR         260818.0      2
           OX40N      258737.0      4
H-LC       X40N       236513.0      3
H-MC       X40        210417.0      4
H-LC       AR         200115.0      2
H-MC       XY25       179736.0      2
L-SC       XR         172105.0      2
M-SC       AR         168074.0      2
M-LC       XY24       158602.0      2
M-SC       XY25       152698.0      1
           OX40N      141425.0      4
           XR         129987.0      2
M-LC       XR         110676.0      1
H-LC       XY25       109041.0      3
M-LC       X5K        104230.0      1
H-MC       XR          99679.0      1
L-SC       OX40N       88823.0      1
H-LC       X200        88165.0      1
H-MC       X40N        85167.0      2
M-SC       X40         73042.0      1
H-SC       MH          72642.0      1
H-MC       OX40N       71823.0      1
H-LC       XY24        70696.0      1
L-MC       XR          60490.0      1
L-LC       XY25        43331.0      1
H-MC       MH          14534.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
